In [2]:
# Imports
import pandas as pd

In [107]:
# This is where we load all the data through oracle; but for now I'll just read.csv the data in.

emissions = pd.read_csv("data/tidied_emissions_with_key.csv")
employment = pd.read_csv("data/tidied_employment_estimates.csv")
industry = pd.read_csv("data/tidied_industry_mapping.csv")
population = pd.read_csv("data/tidied_population_estimates.csv")

In [250]:
## model1: emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year

# joining emissions and industry together
emissions_industry = pd.merge(emissions, industry, left_on="industry_code", right_on="IndustryCode")

# filter out canada in emissions_industry dataset
emissions_industry = emissions_industry[emissions_industry["province"] != "Canada"]

# mutate a new column called ProvinceYearDurable in EmissionsIndustry
emissions_industry["ProvinceYearDurable"] = emissions_industry["province"] + emissions_industry["year"].astype(str) + emissions_industry["Durable"].astype(str)

# remove households, put in separate dataframe
emissions_industry_household = emissions_industry[emissions_industry["ioic_code"].isin(['9999999', '9999998'])]
emissions_industry = emissions_industry[~emissions_industry["ioic_code"].isin(['9999999', '9999998'])]

# drop a few duplicate columns
emissions_industry.drop(columns=["IndustryCode", "IndustryName", "IOIC", "industry_name", "industry_code", "NAICS"], inplace=True)

# rename count to "emissions"
emissions_industry.rename(columns={'count': 'emissions'}, inplace=True)

# group by Province, Year, Durable, sum emissions for each industry
grouped_emissions = pd.DataFrame(emissions_industry.groupby(['province', 'year', 'Durable'])['emissions'].sum())
grouped_emissions = grouped_emissions.reset_index()
grouped_emissions["ProvinceYearDurable"] = grouped_emissions["province"] + grouped_emissions["year"].astype(str) + grouped_emissions["Durable"].astype(str)




# remove Manufacturing, Canada from naics_industry
employment_tidy = employment[(employment["naics_industry"] != "Manufacturing") & (employment["province"] != "Canada")]

# rename employment estimate data's manufacturing column to 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate a new column called ProvinceYearDurable in employment
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)

# remove Manufacturing, Canada from naics_industry
employment_tidy = employment[(employment["naics_industry"] != "Manufacturing") & (employment["province"] != "Canada")]

# rename employment estimate data's manufacturing column to 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate a new column called ProvinceYearDurable in employment
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)
employment_tidy["employed"] = employment_tidy["count"]
employment_tidy = employment_tidy.drop(columns=["count", "province", "year", "durable"])




# join the grouped_emissions with employment_tidy, drop key
emissions_industry_employment = pd.merge(grouped_emissions, employment_tidy, on="ProvinceYearDurable")
emissions_industry_employment = emissions_industry_employment.drop(columns="ProvinceYearDurable")

# ma

/tmp/ipykernel_156/213820461.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
/tmp/ipykernel_156/213820461.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})


In [251]:
emissions_industry_employment

,province,year,Durable,emissions,employed
0,Alberta,2012,0,27513,50.6
1,Alberta,2012,1,4249,88.8
2,Alberta,2013,0,28619,59.9
3,Alberta,2013,1,4921,82.8
4,Alberta,2014,0,27933,56.5
...,...,...,...,...,...
195,Saskatchewan,2019,1,775,18.4
196,Saskatchewan,2020,0,3011,11.9
197,Saskatchewan,2020,1,762,18.8
198,Saskatchewan,2021,0,3135,12.9


In [ ]:
# join grouped_emissions with employment_tidy
grouped_emissions_employment = pd.merge(grouped_emissions, employment_tidy, on="ProvinceYearDurable")

grouped_emissions_employment = grouped_emissions_employment.reset_index()





# rename several columns
emissions_industry_employment["province"] = emissions_industry_employment["province_x"]
emissions_industry_employment["year"] = emissions_industry_employment["year_x"]
emissions_industry_employment["employed"] = emissions_industry_employment["count"]

# remove some duplicate/unused columns
emissions_industry_employment = emissions_industry_employment.drop(columns=["province_y", "year_y", "durable", "province_x", "year_x", "count"])




# group by province, year, durable, sum emissions
grouped_emissions = pd.DataFrame(emissions_industry_employment.groupby(["province", "year", "Durable"])['emissions'].sum())
grouped_emissions = grouped_emissions.reset_index()
grouped_emissions["ProvinceYearDurable"] = grouped_emissions["province"] + grouped_emissions["year"].astype(str) + grouped_emissions["Durable"].astype(str)


In [220]:
grouped_emissions


emissions
province     year Durable           
Alberta      2012 0            27513
                  1             4249
             2013 0            28619
                  1             4921
             2014 0            27933
...                              ...
Saskatchewan 2019 1              775
             2020 0             3011
                  1              762
             2021 0             3135
                  1              765

[200 rows x 1 columns]

# linear regression: 

models:

1. emissions/population (by year, province) ~ province(dummy) * year ---> households
2. emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year

In [ ]:
# filter out canada in population dataset
population = population[population["Geography"] != "Canada"]

# mutate a new column called ProvinceYear in Population
population["ProvinceYear"] = population["Geography"] + population["year"].astype(str)

# joining emissions+industry with population dataset
emissions_industry_population_data = pd.merge(emissions_industry, population, on="ProvinceYear")

# clean up data
model1_data = emissions_industry_population_data.drop(columns=["year_y", "Geography", "IndustryCode", "IndustryName", "IOIC", ])

In [ ]:
# filter employment data to stop using Canada and Manufacturing
employment_tidy = employment[(employment["province"] != "Canada") & (employment["naics_industry"] != "Manufacturing")]

# rename naics_industry column, redesign column to be called durable in employment data that is 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate employment data to create province+year+durable
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)

# join emissions_industry and employment data
emissions_industry_employment = pd.merge(emissions_industry, employment_tidy, on="ProvinceYearDurable")

# drop a few columns
emissions_industry_employment.drop(columns=["province_y", "year_y", "durable", "IndustryCode", "IndustryName", "IOIC"], inplace=True)

# rename some columns
emissions_industry_employment_rename_dict = {'year_x': "year", "count_x": "emissions", "count_y": "employed"}
emissions_industry_employment.rename(columns=emissions_industry_employment_rename_dict, inplace=True)

# filter out household data to remove household 
emissions_industry_employment = emissions_industry_employment[~emissions_industry_employment["ioic_code"].isin(['9999999', '9999998'])]

# mutate employed to divide by how many industries there are per province; must make an assumption that each industry is equal in size. (not enough data to assume otherwise)
emissions_industry_employment["employed"] = emissions_industry_employment["employed"]/97*1000

# create response variable: 
emissions_industry_employment[emissions_per_employed